In [ ]:
# Import packages
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',95)
pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('precision',4)
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from wordcloud import WordCloud, STOPWORDS


In [ ]:
# Opening the pickled file
f = open('df5.pkl', 'rb')
df5 = pickle.load(f)
f.close()

In [ ]:
# Change item_description to item_name
df5.rename(columns={'item_description' : 'item_name'},inplace=True)

In [ ]:
# Count of different store types
df5.store_type.value_counts()

In [ ]:
all_item_names = df5.item_name.value_counts()
all_item_names.head(50)

### Market Basket Analysis

In [ ]:
# Filter for top 500 item names that accounted for higher volume sales
top_500 = df5.groupby(['item_name']).agg({'volume_sold_liters': 'sum'}).sort_values(
    by = 'volume_sold_liters', ascending= False).reset_index()[0:500]['item_name'].tolist()

In [ ]:
top_500[0:10]

In [ ]:
# Create dataframe using top_50 item name list for MBA 
df_top_500 = df5.loc[df5['item_name'].isin(top_500)]
df_top_500.head()

In [ ]:
# shape of the df_top_50
df_top_500.shape

In [ ]:
# Make sure Egg Nog in list
[s for s in top_500 if s.lower().find('egg nog')>=0 ]

In [ ]:
# is this one item above the only egg nog?
egg_nog_names = [s for s in all_item_names.index.values if s.lower().find('egg nog')>=0 ]
egg_nog_names

In [ ]:
def make_basket(df, store_filter=None, month=None, date_grouping_freq='M', item_filter=None):
  '''
  Make the basket of items bought together and filter by 
      store_filter: 'supermarket', 'retail store', 'convenience store', 'big box', 'drugstore'
      month: 1-12 for month;  January==1, December==12
      date_grouping_freq: 'M' for group by month, 'W' by week
      item_filter: filter by item_name containing the string here. 
  '''
  if store_filter:
    df = df.loc[ df['store_type']==store_filter ]
  if month:
    df = df.loc[ df['date'].dt.month == month ]
  if item_filter:
    df = df.loc[ df['item_name'].str.contains(item_filter) ]
  basket = (df.groupby(
      [pd.Grouper(key='date',freq=date_grouping_freq),
       'store_number','item_name']
       )['item_name'].size().unstack().\
       reset_index().fillna(0).set_index(['date','store_number']))
  def encode_units(x):
      if x <= 0:
          return 0
      if x >= 1:
          return 1
  basket = basket.applymap(encode_units)
  return basket, df


In [ ]:
def plot_most_freq_items(df, label):
  # Most frequent items bar plot
  s = df['item_name'].value_counts().head(20)
  num_items = len(s)
  color = plt.cm.inferno(np.linspace(0,1,num_items))
  plt.rcParams['figure.figsize'] = (10,6)
  s.plot.bar(color = color)
  plt.title(f'Top {num_items} Most Frequent Items in {label}')
  plt.ylabel('Counts')
  plt.xlabel('Items')
  plt.show()

In [ ]:
all_rules = {} # dict to store rules from various scenarios

In [ ]:
# ALL Stores
print('\n\n' + '='*80 + f'\n========== ALL STORES ==========\n' + '='*80 + '\n')
basket, df_filtered = make_basket(df_top_500)
plot_most_freq_items(df_filtered, label='ALL_STORES')
frequent_itemssets = apriori(basket,min_support=0.1,max_len=2,use_colnames=True )
# Creates association rules based on if-then frequency
rules = association_rules(frequent_itemssets,metric='lift')

all_rules['all_stores'] = rules

# Prune rules
#rules = rules[ (rules['confidence'] >= .90) & (rules['support'] >= 0.55) ]

print('\nTop 20 Rules sorted by support') 
print( rules.sort_values('support',ascending = False).head(20) )

print('\nTop 20 Rules sorted by confidence') 
print( rules.sort_values('confidence',ascending = False).head(20) )

print('\nTop 20 Rules sorted by lift') 
print( rules.sort_values('lift',ascending = False).head(20) )

In [ ]:
# Run MBA for each store type
store_types = ['supermarket', 'retail store', 'convenience store', 'big box', 'drugstore']

for s in store_types:
  print('\n\n' + '='*80 + f'\n========== {s} ==========\n' + '='*80 + '\n')

  basket, df_filtered = make_basket(df_top_500, store_filter=s)
  plot_most_freq_items(df_filtered, label=s)
  frequent_itemssets = apriori(basket,min_support=0.1,max_len=2,use_colnames=True )
  # Creates association rules based on if-then frequency
  rules = association_rules(frequent_itemssets,metric='lift')
  all_rules[s] = rules

  # Prune rules
  #rules = rules[ (rules['confidence'] >= .90) & (rules['support'] >= 0.55) ]

  print('\nTop 20 Rules sorted by support\n') 
  print( rules.sort_values('support',ascending = False).head(20) )

  print('\nTop 20 Rules sorted by confidence\n') 
  print( rules.sort_values('confidence',ascending = False).head(20) )

  print('\nTop 20 Rules sorted by lift\n') 
  print( rules.sort_values('lift',ascending = False).head(20) 

In [ ]:
for m in range(1, 13):
  name = 'all_stores_month_' + f'{m:02}'
  print('\n\n' + '='*80 + f'\n========== {name} ==========\n' + '='*80 + '\n')
  basket, df_filtered = make_basket(df_top_500, month=m)
  plot_most_freq_items(df_filtered, label=name)
  frequent_itemssets = apriori(basket,min_support=0.1,max_len=2,use_colnames=True )
  # Creates association rules based on if-then frequency
  rules = association_rules(frequent_itemssets,metric='lift')
  all_rules[name] = rules

  # Prune rules
  #rules = rules[ (rules['confidence'] >= .90) & (rules['support'] >= 0.55) ]

  print('\nTop 20 Rules sorted by support') 
  print( rules.sort_values('support',ascending = False).head(20) )

  print('\nTop 20 Rules sorted by confidence') 
  print( rules.sort_values('confidence',ascending = False).head(20) )

  print('\nTop 20 Rules sorted by lift') 
  print( rules.sort_values('lift',ascending = False).head(20) )

In [ ]:
# how often is egg nog sold?  
# as you see below, not very often compared to the > 15 million rows in the df_top_500
print(f'Total number of rows in main df_top_500: {len(df_top_500):,}\n')
df_top_500[ df_top_500['item_name'].str.contains('egg nog') ]['item_name'].value_counts().plot(kind = 'bar', color= 'red');

In [ ]:
# Only Egg Nog in December
name = 'ALL STORES EGG NOG ONLY IN DECEMBERS ONLY'
print('\n\n' + '='*80 + f'\n========== {name} ==========\n' + '='*80 + '\n')
basket, df_filtered = make_basket(df_top_500, month=12, item_filter=None)
plot_most_freq_items(df_filtered, label=name)
frequent_itemssets = apriori(basket,min_support=0.01,max_len=2,use_colnames=True )
# Creates association rules based on if-then frequency
rules = association_rules(frequent_itemssets,metric='lift')

# Filter for only egg nog
rules = rules[  rules['antecedents'].apply(lambda x: (str(x).find('egg nog'))>=0 ) ]

all_rules[name] = rules

print('\nTop 20 Rules sorted by support') 
print( rules.sort_values('support',ascending = False))

print('\nTop 20 Rules sorted by confidence') 
print( rules.sort_values('confidence',ascending = False).head(20) )

print('\nTop 20 Rules sorted by lift') 
print( rules.sort_values('lift',ascending = False).head(20) )

In [ ]:
# joining all rules
main_df = pd.DataFrame(columns=['antecedents', 'consequents'])
for k, v in all_rules.items():
  k = k.replace(" ", "_")
  temp = v.add_suffix(" " + k)
  temp = temp.rename(columns={ 'antecedents '+k:'antecedents', 'consequents '+k:'consequents'})
  main_df = pd.merge(main_df, temp, on=['antecedents', 'consequents'], how='outer')

In [ ]:
columns = main_df.columns
select_c = [c for c in columns if c.find('lift')==0 ]
select_c = ['antecedents', 'consequents'] + select_c
main_df[select_c].sort_values(by='lift all_stores', ascending=False).head(10)

In [ ]:
def make_wordcloud(df, store_filter=None, month=None):
'''
Make a WordCloud image and filter by
store_filter: 'supermarket', 'retail store', 'convenience store',
'big box', or 'drugstore'. None is all stores
month: 1-12 for month; January==1, December==12
'''
if store_filter:
df = df.loc[ df['store_type']==store_filter ]
if month:
df = df.loc[ df['date'].dt.month == month ]


cloud_super = df['item_name'].value_counts().head(60).to_frame().reset_index()


# create a dictionary with counts of frequency. For example:
# { 'hawkeye vodka' : 140169,
# 'black velvet'. : 131129,
# 'five o'clock vodka': 97057,
# ... }
counter = dict( zip( cloud_super['index'], cloud_super['item_name'] ) )


final_wordcloud = WordCloud(width = 800, height = 800,
background_color ='black',
min_font_size = 8).generate_from_frequencies(counter)


# Plotting the WordCloud
plt.figure(figsize = (10, 10), facecolor = None)
plt.imshow(final_wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
make_wordcloud(df_top_500,'supermarket'

In [ ]:
#### Function to visualize association rules

In [ ]:
import networkx as nx  
def draw_graph(rules, rules_to_show):
  G1 = nx.DiGraph()   
  color_map=[]
  N = 50
  colors = np.random.rand(N)    
      
  for i in range (rules_to_show):      
    for a in rules.iloc[i]['antecedents']:               
        G1.add_nodes_from([a])
        consequent = list(rules.iloc[i]['consequents'])[0]
        G1.add_edge(a, consequent, color=colors[i] , weight = 2)   
 
  for node in G1:
       found_a_string = False
       if found_a_string:
            color_map.append('yellow')
       else:
            color_map.append('green')       
   
  edges = G1.edges()
  colors = [G1[u][v]['color'] for u,v in edges]
  weights = [G1[u][v]['weight'] for u,v in edges]
 
  pos = nx.spring_layout(G1, k=16, scale=1)
  nx.draw( G1, pos, node_color = color_map, edge_color=colors, 
          width=weights, font_size=16, with_labels=False)            
   
  for p in pos:  # raise text positions
           pos[p][1] += 0.07
  nx.draw_networkx_labels(G1, pos)
  plt.show()
      

In [ ]:
print('\n Supermarkets\n\n\n')
rules_super = all_rules['supermarket']



#r = rules_super[(rules_super['confidence'] >= .90) & (rules_super['support'] >= 0.55)].sort_values('lift',ascending = False).head(30)
#print(rules_supermarket.columns)
#r = r[ ['antecedents','consequents'] ].head(10)
draw_graph( rules_super, 20)   

In [ ]:
print('\n Retail Store\n\n\n')
rules_super = all_rules['retail store']
draw_graph( rules_super, 20) 

In [ ]:
print('\n Convenience Stores\n\n\n')
rules_super = all_rules['convenience store']
draw_graph( rules_super, 20) 